In [ ]:
import model
import coco_dataset
import pathlib
import show_utils
import matplotlib.pyplot as plt
from torchvision import transforms
import torch

In [ ]:
coco_dataset = coco_dataset.CocoKeypoints(
    root=str(pathlib.Path("../coco") / "images" / "train2017"),
    annFile=str(
        pathlib.Path("../coco")
        / "annotations"
        / "annotations"
        / "person_keypoints_train2017.json"
    ),
    transform=transforms.Resize((368,368)),
)
i=0

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
if device=="cuda":
    device = "cuda:0"
    model = torch.nn.DataParallel(model.openpose(), device_ids=[0])
    model.load_state_dict(torch.load("save_model_avgPaf.pth"))
    model.eval();
print(device)

In [ ]:
i = 0

In [ ]:
sample = coco_dataset[i]
tf_image, pafs, heatmaps, mask_out, keypoints, image, target = sample
tf_image.unsqueeze_(0)

pred_pafs, pred_heatmaps, _, _ = model(tf_image.to(device)) #input

with torch.no_grad():
    tf_image.squeeze_(0)
    pred_pafs.squeeze_(0)
    pred_heatmaps.squeeze_(0)
    show_utils.show1(tf_image)

    plt.figure()
    show_utils.show_heatmaps_combined(pred_heatmaps.cpu())

    plt.figure()
    show_utils.show_heatmaps(pred_heatmaps.cpu())

    # plt.figure()
    # show_utils.show_pafs_combined(pred_pafs.cpu())

    plt.figure()
    show_utils.show_pafs_quiver_combined(pred_pafs.cpu(), size=(46, 46))

    print(i)
    i+=1